<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Práctica 3 - Enumerativos </h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>
</div>

En ésta práctica utilizaremos el concepto de ramificación y poda para la resolución del siguiente problema.

La construcción de un artefacto requiere ensamblar $M$ piezas que identiﬁcamos con números entre $1$ y $M$. El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas. Los costes, que son valores positivos, se nos proporcionan en una matriz $C$. La celda $c_{ij}$ es el coste de ensamblar la pieza $i$ cuando ya se han ensamblado $j$ piezas. Deseamos calcular el orden de ensamblado de menor coste total, entendido éste como la suma del coste de ensamblaje de cada una de las piezas. 

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>
    </p></div>




<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>Ramificación y poda: Problema del ensamblaje</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
  Implementa mediante ramificación y poda una solución al problema del ensamblaje con tal de encontrar el listado ordenado de piezas cuyo coste total de ensamblaje sea mínimo.
  </p>
  </div>
    </div></div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>C: Matriz de costes de tamaño $M\times M$, donde M es el número de piezas. El elemento $i,j$ de la matriz representa el coste de ensamblaje para la pieza $i$ tras haber colocado $j$ piezas. 
</li>
</ul>
<br>
<h3>OUTPUT</h3>
Lista de números de $1$ a $M$ ordenados de forma que el coste total de ensamblaje sea mínimo. Ej: [5, 3, 2, 4, 1]
</p>

</div>

In [52]:
import heapq
import numpy as np
#init
def branch_bound(C,M):
    """
    Función que obtiene el orden óptimo de las piezas tal que el coste total es mínimo.
    Args:
        - C: matriz de costes M x M
    """
    #matriz donde podremos seleccionar correctamente los items
    matriz=np.empty(shape=(M,M),dtype=bool)
    matriz.fill(True)#matriu on controlarem els visitats

    cota_sup=cota_superior(C)
    heap=[ ] 
    coste_acumulado=0 
    items= [float('inf')]*M 
    millor_solucio=[] 
    v=[] 
    elem=0 
    ctr=0  
    
    '''
        -cota_sup: valor inicial on tindrem el valor que no pot passar
        -matriz: Matriu numpy on marcarem a False on haguem visitat
        -heap: sera la nostra cua de prioritats
        -coste_acumulado: cost que anirem acumulant a cada branca
        -items: llista inicialitzada a inf, on anirem colocant els elements a la posició més econòmica
        -millor_solucio: variable on guardarem una possible millor solucio completa
        -v: llista auxiliar utilitzada en el metode get_cost
        -elem: serà cada element que podem tractar.
        -ctr: es un comptador, la seva funció es donar un ID a cada "node"(tupla) de la cua prioritaria,
            d'aquesta manera, mai arribarem a comparar np array amb np array, ja que fer-ho dóna errors.
    '''
    
    #inicialitzarem la cua fent la primera ramificació, amb l'element 0
    for i in range(M):
        cost=get_cost(C,M,elem,i,v.copy())#minimo fila + coste objeto
        coste_acumulado=C[elem][i] #coste de la rama, se acumulará posteriormente
        it_private=items.copy() #farem una còpia dels items per a cada estat
        it_private[i]=elem # i hi afegirem l'element
        matriz_n=matriz.copy()
        matriz_n=fill_matrix(M,i,elem,matriz_n)#farem copia de la matriu de visitats i canviarem a false els valors visitats
        ctr+=1 #sumem 1 al nostre id de cada estat
        heap.append((cost,coste_acumulado,elem,it_private,i,ctr,matriz_n))#els posem a la cua
        coste_acumulado=0
        
    heapq.heapify(heap)
    
    
    while heap:
        actual, acc, elemento, its, col,_ctr,mat = heapq.heappop(heap)
        #expulsem de la cua un "node" que serà el nostre estat actual
        if actual>cota_sup:
            #si el valor que hem obtingut, és més gran que la cota superior, fem la primera poda.
            continue
        else:
            #altrament, comencem a tractar el següent element
            _=elemento+1
            #si no tenim cap inf als items d'aquest estat i el valor actual es menor que la cota_sup.
            if float('inf') not in its and actual<cota_sup:
                millor_solucio=its #marquem que de moment, la millor solucio serà els items d'aquest estat
                cota_sup=acc #i que la cota superior serà el valor acumulat de les branques
            else:
                #altrament, iterarem sobre les posicions que no hem visitat encara
                for q in range(M):
                    #si aquesta posicio no ha estat visitada, tractarem aquest nou cas
                    if mat[_][q]:
                        _acc=C[_][q]+acc #acumulem el valor de la branca anterior a aquesta branca
                        _actual=get_cost(C,M,_,q,v.copy())+_acc #sumem al valor de get_cost la suma anterior
                        _its=its.copy() #fem còpia dels items i de la matriu de visitats
                        _its[q]=_#posem l'element a la seva posició més òptima
                        _mat=mat.copy()
                        _mat=fill_matrix(M,q,_,_mat)#omplenem la combinació
                        ctr+=1
                        heapq.heappush(heap,(_actual,_acc,_,_its,ctr,q,_mat)) #ho posem a la cua
    return millor_solucio,cota_sup
                    

#calcula els costos parcials quan no hem agafat tots els elements
def get_cost(C,M,obj,col,visited):
    cost=C[obj][col] 
    j_temporal=0
    less=float('inf')

    for i in range(obj+1,len(C)):
        for j in range(len(C)):
            if j not in visited:
                if C[i][j]<less and j!=col:
                    less=C[i][j]
                    j_temporal=j
        visited.append(j_temporal)
        cost+=less
        less=float('inf')

    return cost                
              
#amb aquesta funcio, omplenem de False la columna/fila ja visitats d'aquell estat en concret         
def fill_matrix(M,t,elem,matrix):
    for i in range(M):
        matrix[elem][i]=False
        matrix[i][t]=False
        
    return matrix

#Cogemos el menor de cada fila
def cota_inferior(C):
    inferior=0
    for i in C.keys():
        less=min(C[i].values())
        inferior+=less
    return inferior
#Cogemos los valores de la diagonal
def cota_superior(C):
    superior=0
    for i in C.keys():
        diag=C[i][i]
        superior+=diag
    return superior

In [53]:
import random
import time

from random import randint
random.seed(1)
M = 10 # Número de piezas
# Matriz de costes generada aleatoriamente
C = {}
for i in range(M):
    C[i] = {}
    for j in range(M):
        C[i][j] = randint(1, 25)
        
    print(i,C[i])
#print(C)


t = time.process_time()


solucion = branch_bound(C ,10)
print(solucion)
elapsed_time = time.process_time() - t
print(elapsed_time)

0 {0: 5, 1: 19, 2: 25, 3: 3, 4: 9, 5: 4, 6: 16, 7: 25, 8: 15, 9: 16}
1 {0: 21, 1: 13, 2: 7, 3: 4, 4: 16, 5: 1, 6: 13, 7: 14, 8: 20, 9: 25}
2 {0: 25, 1: 1, 2: 23, 3: 15, 4: 9, 5: 24, 6: 8, 7: 19, 8: 4, 9: 11}
3 {0: 1, 1: 1, 2: 1, 3: 21, 4: 18, 5: 1, 6: 13, 7: 22, 8: 7, 9: 14}
4 {0: 24, 1: 1, 2: 17, 3: 8, 4: 25, 5: 15, 6: 16, 7: 18, 8: 8, 9: 12}
5 {0: 8, 1: 22, 2: 8, 3: 25, 4: 15, 5: 10, 6: 1, 7: 14, 8: 18, 9: 21}
6 {0: 4, 1: 6, 2: 21, 3: 24, 4: 10, 5: 4, 6: 24, 7: 11, 8: 24, 9: 23}
7 {0: 17, 1: 14, 2: 17, 3: 22, 4: 7, 5: 10, 6: 10, 7: 19, 8: 16, 9: 17}
8 {0: 13, 1: 19, 2: 2, 3: 16, 4: 8, 5: 24, 6: 13, 7: 14, 8: 22, 9: 6}
9 {0: 12, 1: 18, 2: 23, 3: 25, 4: 22, 5: 24, 6: 12, 7: 3, 8: 15, 9: 22}
([6, 4, 3, 0, 7, 1, 5, 9, 2, 8], 31)
0.0


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios </h3></p> </center> 

</div>

### En esta sección debéis responder a las preguntas siguientes:
#### 1. Explica cómo has realizado tu implementación, enfatizando en la heurística empleada.

Principalmente sigo la estrategia que se ve en el pdf de enumerativos, como explico en los comentarios, mi idea inicial fue ir recortando o ignorando los objetos y columnas ya visitadas.

Primera parte:
Sirve para inicializar la cola, solo con el primer elemento, para conseguir la primera ramificación.

Segunda parte:

Mientras el heap no este vacío, voy desencolando y tratando estados. Si el valor "actual"( el que se calcula seleccionando a--A y luego los minimos sin importar si repites fila) supera la cota superior se va a podar ese estado y no vamos a incluir sus derivados a la cola.
Si da el caso que la solucion(items) es completa y el valor es menor a la cota superior, pues vamos a dejar "reservado" ese estado como la mejor solución. Hasta que llegue otro, o no.

Si no es completo, vamos a crear los estados derivados del estado actual, donde no podremos volver a crear estados que ya hemos visitado dentro de ese estado, al tener cada "nodo" una matriz de visitados, ahi se indicara cual esta por visitar y cual no. Esos estados los vamos a encolar.

Funciones auxiliares:

Tengo dos funciones que calculan las cotas, solo uso la cota superior.
La funcion get_cost es la que a partir de el elemento en concreto, te ajusta unos limites y te hace la suma de un coste parcial. Como he dicho antes, tenemos a--->A, tanto A como a estaran "tachados" y solo va a coger los minimos de las restantes.
La funcion fill_matrix es para rellenar de False las combinaciones ya visitadas.





#### 2. Análisis de complejidad de la implementación

Haciendo ramificación y poda mejoramos mucho la complejidad. Pero sigue siendo una búsqueda exhaustiva, es decir la complejidad que vamos a tener que fijar va a ser la del peor caso, aunque se de en una probabilidad muy baja.
La complejidad de mi algoritmo es O(n^2)


<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>

<p>
La entrega tiene que consistir en un solo fichero **.ipynb** con el nombre: *Apellido1_Apellido2*_*Nombre*_ASS3.ipyn
</p>
</div>